In [433]:
import pandas as pd
import re
df = pd.read_excel('pd练习：008：多级表转一维表-子目.xlsx', usecols='A:B')
df.head(10)

,子目号,子目名称
0,604-02,拆除交通标志立柱（含基础，如果有）
1,604-02-01,单柱式
2,604-02-02,双柱式
3,604-02-03,悬臂式
4,604-02-03-1,单悬臂式
5,604-02-03-2,双悬臂式
6,604-02-04,门架式
7,604-03,新增交通标志面板
8,604-03-01,单柱式
9,604-03-01-1,1260×1200


In [431]:
(
    df1 := df.apply(
        lambda x: pd.Series([x.子目名称], index=[len(re.findall('-', x.子目号))]),
        axis=1,
    )
    .set_axis(list('abc'), axis=1)
    .set_index(
        pd.MultiIndex.from_frame(
            df.子目号.str.extract(r'(((\d+-\d+)(?:-\d+)?)(?:-\d+)?)')
            .iloc[:, ::-1]
            .apply(lambda x: x.drop_duplicates(ignore_index=True), axis=1),
            names=['level_0', 'level_1', 'level_2'],
        )
    )
    .assign(
        a=lambda x: x.a.ffill(),
        b=lambda x: x.b.ffill(),
    )
)[~((df1[['a', 'b']] == df1[['a', 'b']].shift(-1)).all(axis=1) & (df1.c.isna()))]

a    b                   c
level_0 level_1   level_2                                                
604-02  NaN       NaN          拆除交通标志立柱（含基础，如果有）  NaN                 NaN
        604-02-01 NaN          拆除交通标志立柱（含基础，如果有）  单柱式                 NaN
        604-02-02 NaN          拆除交通标志立柱（含基础，如果有）  双柱式                 NaN
        604-02-03 604-02-03-1  拆除交通标志立柱（含基础，如果有）  悬臂式                单悬臂式
                  604-02-03-2  拆除交通标志立柱（含基础，如果有）  悬臂式                双悬臂式
        604-02-04 NaN          拆除交通标志立柱（含基础，如果有）  门架式                 NaN
604-03  NaN       NaN                   新增交通标志面板  门架式                 NaN
        604-03-01 604-03-01-1           新增交通标志面板  单柱式           1260×1200
                  604-03-01-2           新增交通标志面板  单柱式           1700×1200
                  604-03-01-3           新增交通标志面板  单柱式  2000×3000+1600×820
                  604-03-01-4           新增交通标志面板  单柱式  2000×3000+1800×920
                  604-03-01-5           新增交通标志面板  单柱式  2000×3000+2200×920
                  604-03-01-6           新增交通标志面板  单柱式           2000×3000
        604-03-02 604-03-02-1           新增交通标志面板  双柱式           3780×3600
                  604-03-02-2           新增交通标志面板  双柱式           4410×4200
                  604-03-02-3           新增交通标志面板  双柱式  3600×3600+1600×820
                  604-03-02-4           新增交通标志面板  双柱式  3600×3600+1800×920
                  604-03-02-5           新增交通标志面板  双柱式  3600×3600+2200×920
                  604-03-02-6           新增交通标志面板  双柱式  2000×3000+1600×820
                  604-03-02-7           新增交通标志面板  双柱式  2000×3000+1800×920
                  604-03-02-8           新增交通标志面板  双柱式  2000×3000+2200×920
                  604-03-02-9           新增交通标志面板  双柱式           2000×3000